In [31]:
# Running for the first time? Install Keras:
#!pip install keras
#!pip install tensorflow-gpu --yes
#!pip install opencv-python

In [3]:
import sklearn
import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import pandas as pd
from tensorflow.keras.applications.resnet50 import ResNet50
#from tensorflow.keras.applications.inception_v3 import InceptionV3
# MobileNet?
import cv2
from PIL import Image
import pandas as pd
from os import listdir
from os.path import isfile, join

In [4]:
## Find name of 
import requests
def return_name(wnid):
    name = 'http://www.image-net.org/api/text/wordnet.synset.getwords?wnid='+wnid
    eng_name = requests.get(name).content.decode("utf-8").strip('\n')
    return eng_name

In [31]:
def read_process_image(path, transformation):
    """
    Reads an image and performs the desired transformation
    """
    img = cv2.imread(path,cv2.IMREAD_COLOR)
    
    if img is None:
        return None, None
    rows,cols, channels = img.shape

    if transformation == 'translate':
        x_translate = 1
        y_translate = 1
        M = np.float32([[1,0,x_translate],[0,1,y_translate]]) # translation
        res = cv2.warpAffine(img,M,(cols,rows))
        
    if transformation == 'scale':
        scale_x = 1 # Reduce x by 1 pixel
        scale_y = 1
        res = cv2.resize(img,(cols - scale_x, rows - scale_y), interpolation = cv2.INTER_LINEAR)
        
    if transformation == 'rotate_180': # Rotates 180 deg
        res = cv2.rotate(img, cv2.ROTATE_180)
        
    if transformation == 'rotate_90': # Rotates 90 deg
        res = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
        
    if transformation == 'horizontal_flip':
        res = cv2.flip(img, 1)


    res = cv2.cvtColor(res, cv2.COLOR_BGR2RGB)
    res = Image.fromarray(res)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(img)
    return img, res

In [32]:
def preds_comparison(pred1, pred2):
    """
    Calculates metrics based on two sets of predictions.
    """
    k = len(pred1)
    if k < 1:
        return
    if pred1[0][0] == pred2[0][0]:
        top1_changed = 0
    else:
        top1_changed = 1
        
    top1_delta = abs(pred1[0][2] - pred2[0][2])
    
    avg_delta = abs(sum([abs(pred1[i][2] - pred2[i][2]) for i in range(k)]) / k)
    return top1_changed, top1_delta, avg_delta

In [33]:
def predict(model, img, img_transformed):
    """
    Given a model, an image and a transformed image, generate the predictions
    """
    k = 3
    # Original
    x = np.expand_dims(img, axis=0)
    x = preprocess_input(x)

    preds = model.predict(x)
    #print('Original:', decode_predictions(preds, top=3)[0])
    pred_orig = decode_predictions(preds, top=k)[0]
    top1_class_orig_pred = pred_orig[0][1]
    
    # Transformed 
    x = np.expand_dims(img_transformed, axis=0)
    x = preprocess_input(x)

    preds = model.predict(x)
    #print('Transformed:', decode_predictions(preds, top=k)[0])
    pred_trans = decode_predictions(preds, top=k)[0]
    top1_class_trans_pred = pred_trans[0][1]
    
    top1_changed, top1_delta, avg_delta = preds_comparison(pred_orig, pred_trans)
    return top1_changed, top1_delta, avg_delta, top1_class_orig_pred, top1_class_trans_pred

In [34]:
model = ResNet50(weights='imagenet')

In [38]:
output_df = pd.DataFrame([],
                   columns=['Class Name', 'Orig Pred Class Name', 'Trans Pred Class Name',
                            'Top 1 Changed', 'Top 1 Delta', 'Avg Delta', 'Path'])



wnid = 'n03394272'
folder_path = 'images/{}/'.format(wnid)
class_nm = return_name(wnid)
image_paths = [folder_path + f for f in listdir(folder_path) if isfile(join(folder_path, f))]
for image_path in image_paths:
    img, res = read_process_image(image_path, 'rotate_180')
    if img is None:
        continue
    top1_changed, top1_delta, avg_delta, top1_class_orig_pred, top1_class_trans_pred = predict(model, img, res)
    row_append = np.array([class_nm, top1_class_orig_pred, top1_class_trans_pred, top1_changed, top1_delta, avg_delta, image_path])
    output_df = output_df.append(dict(zip(output_df.columns, row_append)), ignore_index=True)
    
output_df = output_df.astype(dtype= {"Top 1 Changed":"int64",
        "Top 1 Delta":"float64","Avg Delta":"float64"})

In [39]:
output_df.loc[lambda f: f['Top 1 Changed'] == 1] # All results that changed

,Class Name,Orig Pred Class Name,Trans Pred Class Name,Top 1 Changed,Top 1 Delta,Avg Delta,Path
1,freight liner\nliner train,electric_locomotive,passenger_car,1,0.562875,0.229765,images/n03394272/10.jpg
3,freight liner\nliner train,electric_locomotive,freight_car,1,0.288432,0.228894,images/n03394272/12.jpg
4,freight liner\nliner train,tow_truck,chain_saw,1,0.418174,0.163287,images/n03394272/13.jpg
5,freight liner\nliner train,freight_car,monitor,1,0.118373,0.063300,images/n03394272/14.jpg
7,freight liner\nliner train,bullet_train,monitor,1,0.083654,0.070700,images/n03394272/3.jpg
8,freight liner\nliner train,freight_car,barn,1,0.688936,0.242814,images/n03394272/4.jpg
10,freight liner\nliner train,freight_car,television,1,0.230726,0.148500,images/n03394272/6.jpg
11,freight liner\nliner train,freight_car,crane,1,0.115005,0.103606,images/n03394272/7.jpg
13,freight liner\nliner train,garbage_truck,snowplow,1,0.189531,0.122666,images/n03394272/9.jpg


In [40]:
output_df # All results

,Class Name,Orig Pred Class Name,Trans Pred Class Name,Top 1 Changed,Top 1 Delta,Avg Delta,Path
0,freight liner\nliner train,trailer_truck,trailer_truck,0,0.789005,0.266563,images/n03394272/1.jpg
1,freight liner\nliner train,electric_locomotive,passenger_car,1,0.562875,0.229765,images/n03394272/10.jpg
2,freight liner\nliner train,trailer_truck,trailer_truck,0,0.647590,0.226977,images/n03394272/11.jpg
3,freight liner\nliner train,electric_locomotive,freight_car,1,0.288432,0.228894,images/n03394272/12.jpg
4,freight liner\nliner train,tow_truck,chain_saw,1,0.418174,0.163287,images/n03394272/13.jpg
5,freight liner\nliner train,freight_car,monitor,1,0.118373,0.063300,images/n03394272/14.jpg
6,freight liner\nliner train,recreational_vehicle,recreational_vehicle,0,0.542404,0.191584,images/n03394272/2.jpg
7,freight liner\nliner train,bullet_train,monitor,1,0.083654,0.070700,images/n03394272/3.jpg
8,freight liner\nliner train,freight_car,barn,1,0.688936,0.242814,images/n03394272/4.jpg
9,freight liner\nliner train,electric_locomotive,electric_locomotive,0,0.536877,0.241097,images/n03394272/5.jpg
